<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Dueling_ddqn_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
installsCompleted = False
if not installsCompleted:
    %pip install gym
    print("gym is installed", "\n")
    %pip install coremltools
    print("coremltools is installed", "\n")
    %pip install torch 
    print("torch is installed", "\n")
    %pip install torchvision
    print("torchvision is installed", "\n")
    %pip install tensorflow==2.1.0rc1
    print("tensorflow 2.1.0rc1 is installed", "\n")
    %pip install matplotlib
    print("matplotlib is installed", "\n")
    print("All installs completed", "\n")
    clear_output()
else:
    print("All installs completed", "\n")

In [0]:
import os
import numpy as np
import torch as T
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim


class ReplayBuffer():
    
    def __init__(self, max_size, input_shape):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape),
                                     dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_shape),
                                         dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int64)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=uint32)
    
    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = new_state_memory
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, states_, terminal


class DuelingDeepQNetwork(nn.Module):
    def __init__(self, n_actions, name, input_dims, chkpt_dir):
        self.chkpt_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.chkpt_dir, name)
        self.fc1 = nn.Linear(*input_dims, 512)
        self.V = nn.Linear(512, 1)
        self.A = nn.Linear(512, n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        flat1 = F.relu(self.fc1(state))
        V = self.V(flat1)
        A = self.A(flat1)

        return V, A
    
    def save_checkpoint(self):
        print("saving checkpoint")
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(file):
        print("loading checkpoint")
        self.load_state_dict(T.load(self.checkpoint_file))
#   Xcode 7g/Python/DuelingDDQN
class Agent():
    def __init__(self, gamma, epislon, lr, n_actions, input_dims,
                 mem_size, batch_size, eps_min = 0.01, eps_dec=5e-7,
                 replace=1000, chkpt_dir='~/Desktop'):
        
        self.gamma = gamma
        self.epislon = epislon
        self.lr = lr
        self.n_actions = n_actions
        self.input_dims = input_dims
        self.mem_size = mem_size
        self.batch_size = batch_size

        self.eps_min = eps_min
        self.eps_dec = eps_dec
        self.replace_target_cnt = replace
        self.action_space = [i for i in range(self.n_actions)]

        self.memory = ReplayBuffer(mem_size, input_dims)

        self.q_eval = DuelingDeepQNetwork(self.lr, self.n_actions,
                                          input_dims=self.input_dims,
                                          name='lunar_lander_dueling_ddqn_q_eval',
                                          chkpt_dir=self.chkpt_dir)
        
        self.q_next = DuelingDeepQNetwork(self.lr, self.n_actions,
                                          input_dims=self.input_dims,
                                          name='lunar_lander_dueling_ddqn_q_next',
                                          chkpt_dir=self.chkpt_dir)
        
    def choose_action(self, observation):
        if np.random.random() > self.epislon:
            state = T.tensor([observation] , dtype=T.float) .to(self.q_eval.device)
            _, advantage = self.q_eval.forward(state)
            action = T.argmax(advantage).item()

        else:
            action = np.random.choice(self.action_space)

    def store_transition(self, state, action, reward, state_, done):
        self.memory.store_transition(state, action, reward, state_, done)

    def replace_target_network(self):
        if self.learn_Step_counter % self.replace_target_cnt == 0:
            self.q_next.load_state_dict(self.q_eval.state_dict())

    def decrement_epsilon(self):
        self.epislon = self.epislon - self.eps_dec \
                        if self.epislon > self.eps_min else self.eps_min

    def save_models(self):
        self.q_eval.save_checkpoint()
        self.q_next.save_checkpoint()

    def load_models(self):
        self.q_eval.load_checkpoint()
        self.q_next.load_checkpoint()

    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return

        self.q_eval.optimizer.zero_grad()

        self.replace_target_network()

        state, action, reward, new_state, done = \
                             self.memory.sample_buffer(self.batch_size)

        states = T.tensor(state).to(self.q_ev.dev) 
        actions = T.tensor(actions).to(self.q_ev.dev)
        dones = T.tensor(done).to(self.q_ev.dev)
        rewards = T.tensor(reward).to(self.q_ev.dev)
        states_ =T.tensor(new_state).to(self.q_ev.dev)

        indices = np.arange(self.batch_size)

        V_s, A_s = self.q_eval.forward(states)
        V_s, A_s = self.q_next.forward(states_)

        V_s_eval, A_s_eval = self.q_eval.forward(states_)

        q_pred = T.add(V_s, 
                       (A_s - A_s.mean(dim=1, keepdim=True))) [indices, actions]
        q_next = T.add(V_s_, (A_s_ - A_s_.mean(dim=1, keepdim=True)))
        q_eval = T.add(V_s_eval, 
                       (A_s_eval - A_s_eval.mean(dim=1, keepdim=True)))
        
        max_actions = T.argmax(q_eval, dim=1)
        q_next[dones] = 0.0
        q_target = rewards + self.gamma * q_next[indices, max_actions]

        loss = self.q_eval.loss(q_target, q_pred).to(self.q_eval.dev)
        loss.backward()
        self.q_eval.optimizer.step()
        self.learn_Step_counter += 1

        self.decrement_epsilon()
        # 41:30